In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score

# Load the Iris dataset
data = load_iris()
X, y = data.data, data.target

# For this example, we'll turn this into a binary classification problem (e.g., classifying "setosa" vs "non-setosa")
y_binary = (y == 0).astype(int)  # Create a binary label (0 = non-setosa, 1 = setosa)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.3, random_state=42)

# Train a classifier (Random Forest)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Get the predicted probabilities and predicted classes on the test set
pred_probs = model.predict_proba(X_test)[:, 1]  # Probability of being in class "1" (setosa)
predictions = model.predict(X_test)

# Define a confidence threshold for uncertainty (e.g., confidence below 0.6 is considered uncertain)
uncertainty_threshold = 0.7
high_confidence_predictions = pred_probs >= uncertainty_threshold

# Calculate recall in the traditional way
traditional_recall = recall_score(y_test, predictions)
print(f"Traditional Recall: {traditional_recall:.2f}")

# Calculate Uncertainty-Adjusted Recall (UAR)
# UAR only counts high-confidence correct predictions in the recall calculation
correct_positive_predictions = (predictions == y_test) & (y_test == 1) & high_confidence_predictions
total_actual_positives = np.sum(y_test == 1)

UAR = np.sum(correct_positive_predictions) / total_actual_positives
print(f"Uncertainty-Adjusted Recall (UAR): {UAR:.2f}")

# Create a DataFrame for tabular display
results_df = pd.DataFrame({
    'True Label': y_test,
    'Predicted Label': predictions,
    'Prediction Probability (Confidence)': pred_probs,
    'High Confidence Prediction': high_confidence_predictions,
    'Correct High Confidence Positive Prediction': correct_positive_predictions
})

# Display the results in a table
print("\nDetailed Prediction Results:\n", results_df)

# Explanation
print("\nExplanation of Metrics:")
print(f"Traditional Recall considers all positive predictions, regardless of confidence: {traditional_recall:.2f}")
print(f"Uncertainty-Adjusted Recall (UAR) only considers positive predictions made with high confidence (above {uncertainty_threshold} threshold), making it more reliable in uncertain cases: {UAR:.2f}")
